In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

results = search.invoke("What is Nifty Index?")

print(results)

The Nifty 50, launched in 1996, is a key stock market index representing the top 50 companies listed on the National Stock Exchange (NSE). Covering diverse sectors like finance, IT, and energy, it accounts for about 65% of the total market capitalisation on the NSE. With a base value of 1,000, the Nifty 50 has become a critical gauge for India's economic performance, offering investors a ... The Nifty 50 Index is one of India's most prominent stock market benchmarks, widely followed by investors, traders, and financial analysts. It reflects the performance of the top 50 large-cap companies listed on the National Stock Exchange (NSE) of India. In India, Nifty is the market index conceived and launched in the year 1996 by the National Stock Exchange (NSE). It contains a host of indices such as Nifty 50, Nifty Bank, Nifty Metal, Nifty IT, and more. These indices are also traded in the Futures and Options segment of NSE. Nifty is the benchmark index of the National Stock Exchange (NSE), re

In [2]:
print(search.name)
print(search.description)
print(search.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


## Custom Tools

In [3]:
from langchain_core.tools import tool

In [4]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [5]:
result = multiply.invoke({"a": 6, "b": 7})
print(result)

42


In [6]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


## Structured_tool and Pydantic

In [7]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel,Field   

In [8]:
class MultiplyInput(BaseModel):
    a :int = Field(required=True, description="The first number to multiply")
    b : int = Field(required=True, description="The second number to multiply")

/var/folders/yr/35cmrm35625bvzmvg6m3qrhw0000gn/T/ipykernel_97102/555859787.py:2: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  a :int = Field(required=True, description="The first number to multiply")
/var/folders/yr/35cmrm35625bvzmvg6m3qrhw0000gn/T/ipykernel_97102/555859787.py:3: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  b : int = Field(required=True, description="The second number to multiply")


In [9]:
def multiply_function(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [10]:
multiply_tool = StructuredTool.from_function(
    func=multiply_function,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [11]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to multiply', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to multiply', 'required': True, 'title': 'B', 'type': 'integer'}}


## Tool Calling

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [13]:
@tool
def multiply(a: int, b: int)-> int:
    """Multiply two numbers."""
    return a*b

In [14]:
multiply.name

'multiply'

In [15]:
multiply.description

'Multiply two numbers.'

In [16]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [17]:
llm = ChatOpenAI(model='gpt-5-nano')

In [18]:
llm_with_tool = llm.bind_tools([multiply])

In [19]:
llm_with_tool.invoke('Hi how are you?')

AIMessage(content='Hi! I’m here and ready to help. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 282, 'prompt_tokens': 130, 'total_tokens': 412, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CtaVzYlOO1mfZrrjszfdLqPQYfNrp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b7f21-2d8d-7940-af92-cb9994f945a3-0', usage_metadata={'input_tokens': 130, 'output_tokens': 282, 'total_tokens': 412, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 256}})

In [20]:
query = HumanMessage('can you multiply 3 with 100')

In [21]:
messages = [query]

In [22]:
messages

[HumanMessage(content='can you multiply 3 with 100', additional_kwargs={}, response_metadata={})]

In [23]:
result = llm_with_tool.invoke(messages)

In [24]:
result

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 133, 'total_tokens': 223, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CtaW38NNm3fby9skLxJj8Jzci5f14', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7f21-3aa0-7ce1-9c84-ffacb799c40b-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 100}, 'id': 'call_Bl1tEjnq8bHE0LBgN5g7MavD', 'type': 'tool_call'}], usage_metadata={'input_tokens': 133, 'output_tokens': 90, 'total_tokens': 223, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

In [25]:
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 100},
  'id': 'call_Bl1tEjnq8bHE0LBgN5g7MavD',
  'type': 'tool_call'}]

In [26]:
messages.append(result)

In [27]:
messages

[HumanMessage(content='can you multiply 3 with 100', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 133, 'total_tokens': 223, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CtaW38NNm3fby9skLxJj8Jzci5f14', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7f21-3aa0-7ce1-9c84-ffacb799c40b-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 100}, 'id': 'call_Bl1tEjnq8bHE0LBgN5g7MavD', 'type': 'tool_call'}], usage_metadata={'input_tokens': 133, 'output_tokens': 90, 'total_tokens': 223, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'ou

In [28]:
tool_result = multiply.invoke(result.tool_calls[0])

In [29]:
tool_result

ToolMessage(content='300', name='multiply', tool_call_id='call_Bl1tEjnq8bHE0LBgN5g7MavD')

In [30]:
messages.append(tool_result)

In [31]:
messages

[HumanMessage(content='can you multiply 3 with 100', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 133, 'total_tokens': 223, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CtaW38NNm3fby9skLxJj8Jzci5f14', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7f21-3aa0-7ce1-9c84-ffacb799c40b-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 100}, 'id': 'call_Bl1tEjnq8bHE0LBgN5g7MavD', 'type': 'tool_call'}], usage_metadata={'input_tokens': 133, 'output_tokens': 90, 'total_tokens': 223, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'ou

In [32]:
llm_with_tool.invoke(messages).content

'300\n\n(3 × 100 = 300)'

In [33]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated
import requests

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency conversion factor between a given base currency and a target currency
    """
    url = f'https://v6.exchangerate-api.com/v6/32b1118bf6d47de44c9948e9/pair/{base_currency}/{target_currency}'

    response = requests.get(url)

    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    given a currency conversion rate this function calculates the target currency value from a given base currency value
    """
    return base_currency_value * conversion_rate


In [34]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [35]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1767312001,
 'time_last_update_utc': 'Fri, 02 Jan 2026 00:00:01 +0000',
 'time_next_update_unix': 1767398401,
 'time_next_update_utc': 'Sat, 03 Jan 2026 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 90.034}

In [36]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [37]:
llm = ChatOpenAI()

In [38]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [39]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [40]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [41]:
ai_message = llm_with_tools.invoke(messages)

In [42]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_nIdnsWou4qJ679V9ZNBTJX5j',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_aOKFWUwzfVSCa4LRth11W3wL',
  'type': 'tool_call'}]